In [11]:
import requests
import sys
import os
import re
import json
import ipaddress
from functools import reduce

In [12]:
DEBUG_MOD = True
SECURITY_FILE = 'default_cleannetwork.json'
INFO = None
CLEAN_LEVEL = {3:'HIGH',2:'PASSABLE',1:'LOW',0:'UNKNOW',-1:'DANGER'}

In [13]:
def get_router_telecom():
    global INFO
    return INFO['asn']['descr']

In [14]:
def get_router_location():
    global INFO
    return INFO['location']['country_code']

In [15]:
def get_router_brand(ip_address):
    nmap_scan_result=os.popen(f'sudo nmap -sV -sU -p 161 --script=snmp-info {ip_address}').read().split('\n')
    # if DEBUG_MOD:
    #     print(nmap_scan_result)
    for i in nmap_scan_result:
        if 'enterprise:' in i :
            print(i.split('enterprise:')[1].strip())
            return i.split('enterprise:')[1].strip()
    print('brand not found')
    return None

In [16]:
def security_level_analysis(isp_name,enterprise_name):
    with open(SECURITY_FILE) as file_input:
        clean_network_list = json.load(file_input)
    # if DEBUG_MOD:
    #     print(clean_network_list)
    if isp_name in clean_network_list['isp']['clean']:
        if enterprise_name in clean_network_list['hdm']['clean']:
            return 3
        if enterprise_name in clean_network_list['hdm']['unclean']:
            return -1
        else:
            return 1
    elif isp_name in clean_network_list['isp']['unclean']:
        return -1
    else:
        if enterprise_name in clean_network_list['hdm']['clean']:
            return 2
        if enterprise_name in clean_network_list['hdm']['unclean']:
            return -1
        else:
            return 0
    

In [17]:
def ip_into_int(ip):
    return reduce(lambda x,y:(x<<8)+y,map(int,ip.split('.')))

def is_internal_ip(ip):
    ip = ip_into_int(ip)
    net_a = ip_into_int('10.255.255.255') >> 24
    net_b = ip_into_int('172.31.255.255') >> 20
    net_c = ip_into_int('192.168.255.255') >> 16
    return ip >> 24 == net_a or ip >>20 == net_b or ip >> 16 == net_c

In [18]:
def main(ip):
    global INFO
    if is_internal_ip(ip):
        level = security_level_analysis('_Private_ip',get_router_brand(ip))
    else:
        INFO = requests.get(f'https://api.incolumitas.com/?q={ip}').json()
        if DEBUG_MOD:
            print(INFO)
        level = security_level_analysis(get_router_telecom(),get_router_brand(ip))
    print(CLEAN_LEVEL[security_level_analysis(get_router_telecom(),get_router_brand(ip))])
    return level

In [19]:
if __name__ == '__main__':
    main('10.159.65.128')

brand not found


sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


TypeError: 'NoneType' object is not subscriptable

In [ ]:
is_internal_ip('10.159.65.128')

True